# Cenaero

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (8, 6)

os.makedirs('../outputs', exist_ok=True)

In [ ]:
# Constants

NUM_SEQUENCES = 121
DATA_PATH = '../data/38Q31TzlO-{}/npz_data/data.npz'
PARAMS_PATH = '../data/38Q31TzlO-{}/Minamo_Parameters-Wall2D.txt'
RANDOM_SEED = 20210831

In [ ]:
# Data loading and parsing methods

def load_data(simulation_ids, recurrent=False):
    
    inputs, targets = [], []
    
    for simulation_id in simulation_ids:

        data = np.load(DATA_PATH.format(simulation_id))

        # Unused
        # T_top = data['T_top']
        # x = data['x']
        # y = data['y']
        # temperatures = data['temperatures']
        
        # Parse parameters
        with open(PARAMS_PATH.format(simulation_id)) as params_file:
            lines = params_file.read().splitlines()
            power = float(lines[0].split(' = ')[1])
            break_time = float(lines[1].split(' = ')[1])

        # Input data
        time = data['time']
        delta = time.copy()
        delta[1:] = time[1:] - time[:-1]
        laser_position = data['laser_position_x']
        laser_power = data['laser_power']
        power = np.full(laser_power.shape, power)
        break_time = np.full(laser_power.shape, break_time)
        if not recurrent:
            input = np.stack([time, laser_position, laser_power, power, break_time], axis=1)
        else:
            input = np.stack([delta, laser_position, laser_power, power, break_time], axis=1)

        # Target data
        target = np.stack([data['T{}'.format(i + 1)] for i in range(6)], axis=1)

        inputs.append(input)
        targets.append(target)
    
    if not recurrent:
        inputs = np.concatenate(inputs, axis=0)
        targets = np.concatenate(targets, axis=0)
        
    else:
        max_len = max(input.shape[0] for input in inputs)
        
        for i, input in enumerate(inputs):
            inputs[i] = np.pad(input, [(0, max_len - input.shape[0]), (0, 0)])
        for i, target in enumerate(targets):
            targets[i] = np.pad(target, [(0, max_len - target.shape[0]), (0, 0)])
        
        inputs = np.stack(inputs, axis=1)
        targets = np.stack(targets, axis=1)
    
    return inputs.astype(np.float32), targets.astype(np.float32)

In [ ]:
# Preview of the data

inputs, targets = load_data(range(1, 8 + 1), recurrent=True)

print('inputs:', inputs.shape)
print('targets:', targets.shape)

## Train, test and validation split

In [ ]:
np.random.seed(RANDOM_SEED)
permutation = np.random.permutation(np.arange(1, NUM_SEQUENCES + 1))
first_split = int(0.7 * NUM_SEQUENCES)
second_split = int(0.85 * NUM_SEQUENCES)
train_sequence_ids = permutation[:first_split]
valid_sequence_ids = permutation[first_split:second_split]
test_sequence_ids = permutation[second_split:]

In [ ]:
train_inputs, train_targets = load_data(train_sequence_ids, recurrent=False)
valid_inputs, valid_targets = load_data(valid_sequence_ids, recurrent=False)
test_inputs, test_targets = load_data(test_sequence_ids, recurrent=False)

## Machine learning models

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, max_depth=None, random_state=RANDOM_SEED, n_jobs=-1)

In [ ]:
rf.fit(train_inputs, train_targets)

In [ ]:
valid_preds = rf.predict(valid_inputs)

In [ ]:
mse_loss = ((valid_preds - valid_targets) ** 2).sum() / valid_preds.shape[0]
print(mse_loss)

### Validation sample sequence

In [ ]:
sample_valid_id = np.random.choice(valid_sequence_ids, size=(1,))
sample_valid_inputs, sample_valid_targets = load_data(sample_valid_id, recurrent=False)

sample_valid_preds = rf.predict(sample_valid_inputs)

In [ ]:
for i in range(6):
    plt.plot(sample_valid_targets[:, i], color='C{}'.format(i), label='Target {}'.format(i))
    plt.plot(sample_valid_preds[:, i], color='C{}'.format(i), linestyle=':', label='Prediction {}'.format(i))
    plt.legend()

### Training sample sequence

In [ ]:
sample_train_id = np.random.choice(train_sequence_ids, size=(1,))
sample_train_inputs, sample_train_targets = load_data(sample_train_id, recurrent=False)

sample_train_preds = rf.predict(sample_train_inputs)

In [ ]:
for i in range(6):
    plt.plot(sample_train_targets[:, i], color='C{}'.format(i), label='Target {}'.format(i))
    plt.plot(sample_train_preds[:, i], color='C{}'.format(i), linestyle=':', label='Prediction {}'.format(i))
    plt.legend()
plt.tight_layout()
plt.show()

## Deep learning models

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_hidden):
        super().__init__()
        
        hidden_layers = []
        for _ in range(num_hidden - 1):
            hidden_layers.append(nn.Linear(hidden_size, hidden_size))
            hidden_layers.append(nn.ReLU())
        
        self.sequential = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            *hidden_layers,
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        return self.sequential(x)

### Convert data array to tensor

In [ ]:
train_inputs = torch.from_numpy(train_inputs)
train_targets = torch.from_numpy(train_targets)
valid_inputs = torch.from_numpy(valid_inputs)
valid_targets = torch.from_numpy(valid_targets)
test_inputs = torch.from_numpy(test_inputs)
test_targets = torch.from_numpy(test_targets)

In [ ]:
num_train = train_inputs.size(0)
num_valid = valid_inputs.size(0)
num_test = test_inputs.size(0)

### Hyperparameters and instantiation

In [ ]:
BATCH_SIZE = 32
HIDDEN_SIZE = 256
NUM_HIDDEN = 2
LEARNING_RATE = 1e-3
NUM_EPOCH_CONVERGENCE = 5

In [ ]:
mlp = MLP(
    input_size=train_inputs.size(1),
    hidden_size=HIDDEN_SIZE,
    output_size=train_targets.size(1),
    num_hidden=NUM_HIDDEN)
opt = optim.Adam(mlp.parameters(), lr=LEARNING_RATE)

model_name = 'mlp-' + '-'.join(str(HIDDEN_SIZE) for _ in range(NUM_HIDDEN))
print(mlp)

### Training

In [ ]:
from copy import deepcopy

lowest_loss, num_epoch_no_improvement = float('inf'), 0
best_weights = deepcopy(mlp.state_dict())
train_losses, valid_losses = [], []
train_after_epoch_losses = []  # TODO: delete this

epoch = 0
while num_epoch_no_improvement <= NUM_EPOCH_CONVERGENCE:
    
    # Training
    permutation = torch.randperm(num_train)
    train_loss = 0.0

    for i in range(0, num_train, BATCH_SIZE):
        indices = permutation[i:i+BATCH_SIZE]
        batch_inputs = train_inputs[indices, :]
        batch_targets = train_targets[indices, :]
        
        batch_preds = mlp(batch_inputs)
        loss = F.mse_loss(batch_preds, batch_targets)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        train_loss += loss.item()

    train_loss /= int(num_train / BATCH_SIZE)
    train_losses.append(train_loss)
    
    epoch += 1
    
    # Training evaluation after epoch
    with torch.no_grad():
        train_preds = mlp(train_inputs)
        train_after_epoch_loss = F.mse_loss(train_preds, train_targets).item()
    
    train_after_epoch_losses.append(train_after_epoch_loss)
    
    # Validation
    with torch.no_grad():
        valid_preds = mlp(valid_inputs)
        valid_loss = F.mse_loss(valid_preds, valid_targets).item()
    
    valid_losses.append(valid_loss)
    
    if valid_loss < lowest_loss:
        lowest_loss = valid_loss
        num_epoch_no_improvement = 0
        best_weights = deepcopy(mlp.state_dict())
    else:
        num_epoch_no_improvement += 1
    
    print('Epoch {:03d}: train: {:.4f}, train after epoch: {:.4f}, valid: {:.4f}'.format(epoch, train_loss, train_after_epoch_loss, valid_loss))

mlp.load_state_dict(best_weights)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(train_losses, label='Training loss')
ax.plot(valid_losses, label='validuation loss')
ax.grid()
ax.legend()
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Loss')
plt.tight_layout()
fig.savefig('../outputs/{}.pdf'.format(model_name), transparent=True)

### Training sample sequence

In [ ]:
sample_train_id = np.random.choice(train_sequence_ids, size=(1,))
sample_train_inputs, sample_train_targets = load_data(sample_train_id, recurrent=False)

sample_train_inputs = torch.from_numpy(sample_train_inputs)

with torch.no_grad():
    sample_train_preds = mlp(sample_train_inputs)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.grid()
for i in range(6):
    ax.plot(sample_train_targets[:, i], color='C{}'.format(i), label='Target')
    ax.plot(sample_train_preds[:, i], color='C{}'.format(i), linestyle=':', label='Prediction')
handles, labels = ax.get_legend_handles_labels()
handles = handles[:2]
labels = labels[:2]
ax.legend(handles, labels)
ax.set_xlabel('Time step [-]')
ax.set_ylabel('Temperature [°C]')
plt.tight_layout()
plt.savefig('../outputs/{}-train.pdf'.format(model_name))

### Validation sample sequence

In [ ]:
sample_valid_id = np.random.choice(valid_sequence_ids, size=(1,))
sample_valid_inputs, sample_valid_targets = load_data(sample_valid_id, recurrent=False)

sample_valid_inputs = torch.from_numpy(sample_valid_inputs)

with torch.no_grad():
    sample_valid_preds = mlp(sample_valid_inputs)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.grid()
for i in range(6):
    ax.plot(sample_valid_targets[:, i], color='C{}'.format(i), label='Target')
    ax.plot(sample_valid_preds[:, i], color='C{}'.format(i), linestyle=':', label='Prediction')
handles, labels = ax.get_legend_handles_labels()
handles = handles[:2]
labels = labels[:2]
ax.legend(handles, labels)
ax.set_xlabel('Time step [-]')
ax.set_ylabel('Temperature [°C]')
plt.tight_layout()
plt.savefig('../outputs/{}-valid.pdf'.format(model_name))

### Evaluation on the test set

In [ ]:
with torch.no_grad():
    train_preds = mlp(train_inputs)
    train_loss = F.mse_loss(train_preds, train_targets).item()
with torch.no_grad():
    valid_preds = mlp(valid_inputs)
    valid_loss = F.mse_loss(valid_preds, valid_targets).item()
with torch.no_grad():
    test_preds = mlp(test_inputs)
    test_loss = F.mse_loss(test_preds, test_targets).item()

print('Train set: {:.4f}'.format(train_loss))
print('Validation set: {:.4f}'.format(valid_loss))
print('Test set: {:.4f}'.format(test_loss))

## Recurent architectures

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
np.random.seed(RANDOM_SEED)
permutation = np.random.permutation(np.arange(1, NUM_SEQUENCES + 1))
first_split = int(0.7 * NUM_SEQUENCES)
second_split = int(0.85 * NUM_SEQUENCES)
train_sequence_ids = permutation[:first_split]
valid_sequence_ids = permutation[first_split:second_split]
test_sequence_ids = permutation[second_split:]

In [ ]:
train_inputs, train_targets = load_data(train_sequence_ids, recurrent=True)
valid_inputs, valid_targets = load_data(valid_sequence_ids, recurrent=True)
test_inputs, test_targets = load_data(test_sequence_ids, recurrent=True)

In [ ]:
train_inputs = torch.from_numpy(train_inputs)
train_targets = torch.from_numpy(train_targets)
valid_inputs = torch.from_numpy(valid_inputs)
valid_targets = torch.from_numpy(valid_targets)
test_inputs = torch.from_numpy(test_inputs)
test_targets = torch.from_numpy(test_targets)

In [ ]:
class RNN(nn.Module):
    def __init__(self, cell, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        
        if cell == 'gru':
            self.rnn = nn.GRU(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers)
        elif cell == 'lstm':
            self.rnn = nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers)
        else:
            raise NotImplementedError
            
        self.sequential = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x, h0=None):
        x, hn = self.rnn(x, h0)
        return self.sequential(x), hn

In [ ]:
seq_len = train_inputs.size(0)

num_train = train_inputs.size(1)
num_valid = valid_inputs.size(1)
num_test = test_inputs.size(1)

In [ ]:
BATCH_SIZE = 4
HIDDEN_SIZE = 256
NUM_LAYERS = 1
LEARNING_RATE = 1e-2
NUM_EPOCH_CONVERGENCE = 5

In [ ]:
rnn = RNN(
    cell='gru',
    input_size=train_inputs.size(2),
    hidden_size=HIDDEN_SIZE,
    output_size=train_targets.size(2),
    num_layers=NUM_LAYERS)
opt = optim.Adam(rnn.parameters(), lr=LEARNING_RATE)

model_name = 'rnn-' + '-'.join(str(HIDDEN_SIZE) for _ in range(NUM_LAYERS))
print(rnn)

In [ ]:
from copy import deepcopy

lowest_loss, num_epoch_no_improvement = float('inf'), 0
best_weights = deepcopy(rnn.state_dict())
train_losses, valid_losses = [], []
train_after_epoch_losses = []  # TODO: delete this

epoch = 0
while num_epoch_no_improvement <= NUM_EPOCH_CONVERGENCE:
    
    # Training
    permutation = torch.randperm(num_train)
    train_loss = 0.0

    for i in range(0, num_train, BATCH_SIZE):
        indices = permutation[i:i+BATCH_SIZE]
        batch_inputs = train_inputs[:, indices, :]
        batch_targets = train_targets[:, indices, :]
        
        batch_preds, _ = rnn(batch_inputs)
        loss = F.mse_loss(batch_preds, batch_targets)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        train_loss += loss.item()

    train_loss /= int(num_train / BATCH_SIZE)
    train_losses.append(train_loss)
    
    epoch += 1
    
    # Training evaluation after epoch
    with torch.no_grad():
        train_preds, _ = rnn(train_inputs)
        train_after_epoch_loss = F.mse_loss(train_preds, train_targets).item()
    
    train_after_epoch_losses.append(train_after_epoch_loss)
    
    # Validation
    with torch.no_grad():
        valid_preds, _ = rnn(valid_inputs)
        valid_loss = F.mse_loss(valid_preds, valid_targets).item()
    
    valid_losses.append(valid_loss)
    
    if valid_loss < lowest_loss:
        lowest_loss = valid_loss
        num_epoch_no_improvement = 0
        best_weights = deepcopy(rnn.state_dict())
    else:
        num_epoch_no_improvement += 1
    
    print('Epoch {:03d}: train: {:.4f}, train after epoch: {:.4f}, valid: {:.4f}'.format(epoch, train_loss, train_after_epoch_loss, valid_loss))

rnn.load_state_dict(best_weights)

### Training sample sequence

In [ ]:
sample_train_id = np.random.choice(train_sequence_ids, size=(1,))
sample_train_inputs, sample_train_targets = load_data(sample_train_id, recurrent=True)

sample_train_inputs = torch.from_numpy(sample_train_inputs)

with torch.no_grad():
    sample_train_preds, _ = rnn(sample_train_inputs)

In [ ]:
plt.plot(sample_train_preds[:, 0, :])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.grid()
for i in range(6):
    ax.plot(sample_train_targets[:, 0, i], color='C{}'.format(i), label='Target')
    ax.plot(sample_train_preds[:, 0, i], color='C{}'.format(i), linestyle=':', label='Prediction')
handles, labels = ax.get_legend_handles_labels()
handles = handles[:2]
labels = labels[:2]
ax.legend(handles, labels)
ax.set_xlabel('Time step [-]')
ax.set_ylabel('Temperature [°C]')
plt.tight_layout()
plt.savefig('../outputs/{}-train.pdf'.format(model_name))